In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

In [ ]:
!pip install selenium
!pip install webdriver_manager
# webdriver_manager는 4버전 이상에서 크롬 버전을 자동으로 관리

In [4]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

# Crawling

## 동적크롤링

reference: https://selenium-python.readthedocs.io/index.html

### 설치 방법

```cmd
pip install selenium
```

Chrome Driver: https://chromedriver.chromium.org/downloads

### 사용법

기본적인 사용 방법은 아래와 같음
```python
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
 
driver = webdriver.Chrome(executable_path='chromedriver') # executable_path: chromedriver path
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))    # 4버전 이상
# driver = webdriver.Firefox()

driver.get(url="http://www.naver.com")

driver.close()
```

In [117]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install())) 

In [48]:
driver.get('https://finance.daum.net/domestic/market_cap')

In [9]:
bs = BeautifulSoup(driver.page_source, 'lxml')

In [11]:
bs.select('div.box_contents')

[<div class="box_contents"><div style="display: block;"><table>
 <colgroup>
 <col width="44"/>
 <col width="*"/>
 <col width="84"/>
 <col width="84"/>
 <col width="84"/>
 <col width="84"/>
 <col width="94"/>
 <col width="84"/>
 </colgroup>
 <thead>
 <tr>
 <th class="lAlign">순위</th>
 <th class="lAlign">종목명</th>
 <th class="rAlign">현재가</th>
 <th class="rAlign">전일비</th>
 <th class="rAlign">
 <a class="" data-field-name="changeRate" href="javascript:void(0)">등락률 <i>-</i></a>
 </th>
 <th class="rAlign">
 <a class="" data-field-name="marketCap" href="javascript:void(0)">시가총액 <i>-</i></a>
 </th>
 <th class="rAlign">
 <a class="" data-field-name="listedShareCount" href="javascript:void(0)">상장주식수 <i>-</i></a>
 </th>
 <th class="rAlign last">
 <a class="" data-field-name="foreignOwnRate" href="javascript:void(0)">외국인 <i>-</i></a>
 </th>
 </tr>
 </thead>
 <tbody><tr class="first"><td class="first"><span class="time">1</span></td>
 <td><a class="txt" href="/quotes/A005930">삼성전자</a></td>
 <td>
 <sp

#### Locating Elements

find_element: 조건에 해당하는 하나의 값 반환  
find_elements: 조건에 해당하는 다수의 값 반환

element를 찾는 method는 아래와 같음
- by_ID
- by_xpath
- by_link_text
- by_partial_link_text
- by_name
- by_tag_name
- by_class_name
- by_css_selector

```python
element = driver.find_element(By.XPATH, x_path)
element = driver.find_element_by_xpath(x_path)

element = driver.find_element(By.CSS_SELECTOR, css)
element = driver.find_element_by_css_selector(css)
```

해당 값을 못 찾은 경우 NoSuchElementException 발생

In [44]:
# /html/body/div/div[4]/div[2]/div[1]/div[2]/div[2]/div/div/a[1]  1페이지에서 2페이지에 대한 xpath
# /html/body/div/div[4]/div[2]/div[1]/div[2]/div[2]/div/div/a[2]  2페이지에서 3페이지에 대한 xpath
# /html/body/div/div[4]/div[2]/div[1]/div[2]/div[2]/div/div/a[3]  3페이지에서 4페이지에 대한 xpath

driver.get('https://finance.daum.net/domestic/market_cap')
bs = BeautifulSoup(driver.page_source, 'lxml')

total_stock_info = []

columns = [tag.text.strip().replace(' -', '') for tag in bs.select('div.box_contents')[0].select('tr')[0].select('th')]
stock_info = []
for row in range(1, 31):
    stock_info.append([tag.text.strip() for tag in bs.select('div.box_contents')[0].select('tr')[row].select('td')])
total_stock_info.extend(stock_info)

for page in tqdm(range(1, 10)):
    stock_info = []
    driver.find_element(By.XPATH, f'/html/body/div/div[4]/div[2]/div[1]/div[2]/div[2]/div/div/a[{page}]').click()
    time.sleep(1)
    bs = BeautifulSoup(driver.page_source, 'lxml')

    for row in range(1, 31):
        stock_info.append([tag.text.strip() for tag in bs.select('div.box_contents')[0].select('tr')[row].select('td')])

    total_stock_info.extend(stock_info)

100%|██████████| 9/9 [00:10<00:00,  1.21s/it]


In [45]:
# 71 ~ 76
# /html/body/div/div[4]/div[2]/div[1]/div[2]/div[2]/div/div/a[7]: 71페이지에서 72페이지의 xpath
# /html/body/div/div[4]/div[2]/div[1]/div[2]/div[2]/div/div/a[8]: 71페이지에서 72페이지의 xpath
# /html/body/div/div[4]/div[2]/div[1]/div[2]/div[2]/div/div/a[11]

300

In [49]:
# /html/body/div/div[4]/div[2]/div[1]/div[2]/div[2]/div/div/a[7]  71페이지에서 72페이지에 대한 xpath
# /html/body/div/div[4]/div[2]/div[1]/div[2]/div[2]/div/div/a[8]  72페이지에서 73페이지에 대한 xpath
# /html/body/div/div[4]/div[2]/div[1]/div[2]/div[2]/div/div/a[11]

## 71 page
total_stock_info = []
bs = BeautifulSoup(driver.page_source, 'lxml')

columns = [tag.text.strip().replace(' -', '') for tag in bs.select('div.box_contents')[0].select('tr')[0].select('th')]
stock_info = []
for row in range(1, 31):
    stock_info.append([tag.text.strip() for tag in bs.select('div.box_contents')[0].select('tr')[row].select('td')])
total_stock_info.extend(stock_info)

## 72 page
for page in tqdm(range(7, 12)):
    stock_info = []
    driver.find_element(By.XPATH, f'/html/body/div/div[4]/div[2]/div[1]/div[2]/div[2]/div/div/a[{page}]').click()
    time.sleep(1)
    bs = BeautifulSoup(driver.page_source, 'lxml')
        
    for row in range(1, 31):
        try:
            stock_info.append([tag.text.strip() for tag in bs.select('div.box_contents')[0].select('tr')[row].select('td')])
        except IndexError:
            pass

    total_stock_info.extend(stock_info)

100%|██████████| 5/5 [00:05<00:00,  1.18s/it]


##### Send Keys
```python
driver.find_element_by_xpath(xpath).send_keys(id)
driver.find_element_by_xpath(xpath).send_keys(password)
driver.find_element_by_xpath(xpath).click()
```

외에도 Keys 내의 다양한 커맨드 입력 가능
```
from selenium.webdriver.common.keys import Keys
```
|Command          |Action |
|-----------------|-------|
|Keys.ENTER       |엔터    |
|Keys.RRETURN     |       |
|Keys.SPACE       |스페이스 |
|Keys.ARROW_UP    |화살표   |
|Keys.ARROW_DOWN  |       |
|Keys.ARROW_LEFT  |       |
|Keys.ARROW_RIGHT |       |
|Keys.BACK_SPACE  |지우기   |
|Keys.DELETE      |       |
|Keys.CONTROL     |Ctrl   |
|Keys.ALT         |Alt    |
|Keys.SHIFT       |Shift  |
|Keys.TAB         |Tab    |
|Keys.PAGE_UP     |Page-Up|
|Keys.PAGE_DOWN   |Page-Down|
|Keys.TAB         |Tab    |
|Keys.F1 ~ Keys.F9|F1~F9  |
|Keys.ESCAPE      |ESC    |
|Keys.HOME        |Home   |
|Keys.INSERT      |Insert |
|...              |...    |

In [53]:
driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/div[3]/div/div/form/fieldset/div/input').send_keys('월급')
driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/div[3]/div/div/form/fieldset/div/input').send_keys(Keys.ENTER)

##### Drag and Drop

```python
from selenium.webdriver import ActionChains

element = driver.find_element(By.NAME, "source")
target = driver.find_element(By.NAME, "target")

action_chains = ActionChains(driver)
action_chains.drag_and_drop(element, target).perform()
```

##### Moving between Windows

```python
previous_window = driver.window_handles[0]
new_window = driver.window_handles[1]
driver.switch_to.window(new_window)
```


##### Moving

```python
driver.forward()
driver.back()
```

#### Waiting

Selenium은 실행 중인 driver를 이용하여 정보를 추출  
driver에 데이터 로딩이 완료되지 않을 경우 데이터 수집이 불가  
따라서 페이지 이동 등의 작업이 들어가면 다음 작업이 완료될 때까지 대기해야만 함

##### Time

가장 기본적인 방법으로 대기 시간을 time 모듈을 통해 명시

```python
import time
from selenium import webdriver


driver = webdriver.Chrome(executable_path='chromedriver')
driver.get(url="http://www.naver.com")
time.time(5)
driver.close()
```

##### Implicit Wait

위의 time에서는 2가지 문제가 존재
1. 모든 반응형 동작마다 time을 걸어 웹 로딩 대기
2. time을 5초로 설정했지만 실제 그보다 동작이 빨리 끝날 경우 불필요한 시간 대기

위의 문제를 해결하고자 implicit wait을 이용  
이는 driver의 옵션 설정으로 반응형 동작에서 최대 허용 대기 시간을 설정  
만약 허용 시간 이내에 로드가 완료될 경우 다음 작업을 바로 진행

```python
from selenium import webdriver


driver = webdriver.Chrome(executable_path='chromedriver')
driver.implicitly_wait(time_to_wait=5)
driver.get(url="http://www.naver.com")
driver.close()

```

##### Explicit Wait

implicit wait에서는 driver가 로딩을 대기하는 시간을 설정하여 5초를 설정하였어도 이전에 로드가 완료되면 다음 작업의 진행이 가능하였음  
하지만 위 경우도 문제가 있는데 5초가 지나도 내가 원하는 데이터가 로드가 안 되었을 경우 데이터를 수집할 수 없음  
따라서 특정 값이 로드될 때까지 기다렸다가 그 값이 로드되면 다음 작업을 진행할 필요성이 발생  
이 때 이용하는 게 explicit wait

```python
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome(executable_path='chromedriver') 
driver.get(url="http://www.naver.com")

try:
    element = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.CLASS_NAME , 'paging'))
    )
finally:
    driver.quit()

driver.close()

```

until: 조건이 False인 동안에 계속 실행  
not_until: 조건이 True인 동안에 계속 실행

In [59]:
response = requests.get('https://www.egroup.go.kr/egps/wi/stat/kap/psitnCmpnySumryList.do')
bs = BeautifulSoup(response.text, 'lxml')
bs.select('table')

[<table>
 <caption>차트출력조건</caption>
 <colgroup>
 <col class="w15"/>
 <col class="w85"/>
 </colgroup>
 <tbody>
 <tr>
 <th scope="row"><label for="chartTypeSelect">차트유형</label></th>
 <td>
 <li>
 <select id="chartTypeSelect" onchange="javascript:gfn_ps_chartTypeEvent(this.value);">
 <option value="line">line</option>
 <option selected="selected" value="bar">bar</option>
 <option value="spline">spline</option>
 <option value="area">area</option>
 </select>  
 										<input id="patternView" name="patternView" type="checkbox" value="Y"/>
 <label for="patternView">패턴보기</label>
 </li>
 </td>
 </tr>
 <tr>
 <th scope="row">출력 데이터</th>
 <td>
 <ul class="checkbox_form" id="chartPData"></ul>
 </td>
 </tr>
 <tr>
 <th scope="row">X축 데이터</th>
 <td>
 <ul class="checkbox_form" id="chartXData"></ul>
 </td>
 </tr>
 </tbody>
 </table>]

In [60]:
response = requests.get('https://www.egroup.go.kr/egps/ps/io/kap/selectAjaxPsitnCmpnySumryList.do')
bs = BeautifulSoup(response.text, 'lxml')
bs.select('table')

[<table style="width:2330px;">
 <caption>소속회사 개요</caption>
 <colgroup>
 <col width="80px"/>
 <col width="100px"/>
 <col width="260px"/>
 <col width="80px"/>
 <col width="100px"/>
 <col width="100px"/>
 <col width="80px"/>
 <col width="300px"/>
 <col width="80px"/>
 <col width="100px"/>
 <col width="100px"/>
 <col width="100px"/>
 <col width="100px"/>
 <col width="100px"/>
 <col width="100px"/>
 <col width="100px"/>
 <col width="100px"/>
 <col width="100px"/>
 <col width="100px"/>
 <col width="150px"/>
 </colgroup>
 <thead>
 <tr>
 <th scope="colgroup">공개년월</th>
 <th scope="colgroup">기업집단명</th>
 <th scope="colgroup">소속회사명</th>
 <th scope="colgroup">대표자</th>
 <th scope="colgroup">설립일</th>
 <th scope="colgroup">계열편입일</th>
 <th scope="colgroup">업종코드</th>
 <th scope="colgroup">영위업종</th>
 <th scope="colgroup">종업원수</th>
 <th scope="colgroup">결산기일</th>
 <th scope="colgroup">결산주총일</th>
 <th scope="colgroup">기업공개일</th>
 <th scope="colgroup">자산총액</th>
 <th scope="colgroup">부채총액</th>
 <th scope="co

In [66]:
driver.get('https://www.egroup.go.kr/egps/wi/stat/kap/psitnCmpnySumryList.do')
BeautifulSoup(driver.page_source, 'lxml').select('table')

[<table>
 <caption>차트출력조건</caption>
 <colgroup>
 <col class="w15"/>
 <col class="w85"/>
 </colgroup>
 <tbody>
 <tr>
 <th scope="row"><label for="chartTypeSelect">차트유형</label></th>
 <td>
 <li>
 <select id="chartTypeSelect" onchange="javascript:gfn_ps_chartTypeEvent(this.value);">
 <option value="line">line</option>
 <option selected="selected" value="bar">bar</option>
 <option value="spline">spline</option>
 <option value="area">area</option>
 </select>  
 										<input id="patternView" name="patternView" type="checkbox" value="Y"/>
 <label for="patternView">패턴보기</label>
 </li>
 </td>
 </tr>
 <tr>
 <th scope="row">출력 데이터</th>
 <td>
 <ul class="checkbox_form" id="chartPData"></ul>
 </td>
 </tr>
 <tr>
 <th scope="row">X축 데이터</th>
 <td>
 <ul class="checkbox_form" id="chartXData"></ul>
 </td>
 </tr>
 </tbody>
 </table>]

In [65]:
driver.get('https://www.egroup.go.kr/egps/wi/stat/kap/psitnCmpnySumryList.do')
time.sleep(5)
BeautifulSoup(driver.page_source, 'lxml').select('table')

[<table style="width:2330px;">
 <caption>소속회사 개요</caption>
 <colgroup>
 <col width="80px"/>
 <col width="100px"/>
 <col width="260px"/>
 <col width="80px"/>
 <col width="100px"/>
 <col width="100px"/>
 <col width="80px"/>
 <col width="300px"/>
 <col width="80px"/>
 <col width="100px"/>
 <col width="100px"/>
 <col width="100px"/>
 <col width="100px"/>
 <col width="100px"/>
 <col width="100px"/>
 <col width="100px"/>
 <col width="100px"/>
 <col width="100px"/>
 <col width="100px"/>
 <col width="150px"/>
 <col width="150px"/>
 </colgroup>
 <thead>
 <tr>
 <th scope="colgroup">공개년월</th>
 <th scope="colgroup">기업집단명</th>
 <th scope="colgroup">소속회사명</th>
 <th scope="colgroup">대표자</th>
 <th scope="colgroup">설립일</th>
 <th scope="colgroup">계열편입일</th>
 <th scope="colgroup">업종코드</th>
 <th scope="colgroup">영위업종</th>
 <th scope="colgroup">종업원수</th>
 <th scope="colgroup">결산기일</th>
 <th scope="colgroup">결산주총일</th>
 <th scope="colgroup">기업공개일</th>
 <th scope="colgroup">자산총액</th>
 <th scope="colgroup">부채

In [70]:
driver.implicitly_wait(10)

In [69]:
driver.get('https://www.egroup.go.kr/egps/wi/stat/kap/psitnCmpnySumryList.do')
driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div[3]/div/form[2]/div/div[1]/div/div/div[2]/a[3]')

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[2]/div[3]/div[3]/div/form[2]/div/div[1]/div/div/div[2]/a[3]"}
  (Session info: chrome=127.0.6533.120); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00628923+23283]
	(No symbol) [0x005EE934]
	(No symbol) [0x00520733]
	(No symbol) [0x0056326F]
	(No symbol) [0x005634AB]
	(No symbol) [0x0059EE42]
	(No symbol) [0x00584464]
	(No symbol) [0x0059CB8D]
	(No symbol) [0x005841B6]
	(No symbol) [0x00558017]
	(No symbol) [0x0055890D]
	GetHandleVerifier [0x0071A5F3+1013699]
	GetHandleVerifier [0x00723E4C+1052700]
	GetHandleVerifier [0x0071D4B4+1025668]
	GetHandleVerifier [0x0064EA2B+179195]
	(No symbol) [0x005F6833]
	(No symbol) [0x005F3198]
	(No symbol) [0x005F3337]
	(No symbol) [0x005EB4BE]
	BaseThreadInitThunk [0x75E17BA9+25]
	RtlInitializeExcepti

In [71]:
driver.get('https://www.egroup.go.kr/egps/wi/stat/kap/psitnCmpnySumryList.do')
WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div[3]/div[3]/div/form[2]/div/div[1]/div/div/div[2]/a[3]'))
)
bs = BeautifulSoup(driver.page_source, 'lxml')
bs.select('table')

[<table style="width:2330px;">
 <caption>소속회사 개요</caption>
 <colgroup>
 <col width="80px"/>
 <col width="100px"/>
 <col width="260px"/>
 <col width="80px"/>
 <col width="100px"/>
 <col width="100px"/>
 <col width="80px"/>
 <col width="300px"/>
 <col width="80px"/>
 <col width="100px"/>
 <col width="100px"/>
 <col width="100px"/>
 <col width="100px"/>
 <col width="100px"/>
 <col width="100px"/>
 <col width="100px"/>
 <col width="100px"/>
 <col width="100px"/>
 <col width="100px"/>
 <col width="150px"/>
 <col width="150px"/>
 </colgroup>
 <thead>
 <tr>
 <th scope="colgroup">공개년월</th>
 <th scope="colgroup">기업집단명</th>
 <th scope="colgroup">소속회사명</th>
 <th scope="colgroup">대표자</th>
 <th scope="colgroup">설립일</th>
 <th scope="colgroup">계열편입일</th>
 <th scope="colgroup">업종코드</th>
 <th scope="colgroup">영위업종</th>
 <th scope="colgroup">종업원수</th>
 <th scope="colgroup">결산기일</th>
 <th scope="colgroup">결산주총일</th>
 <th scope="colgroup">기업공개일</th>
 <th scope="colgroup">자산총액</th>
 <th scope="colgroup">부채

In [ ]:
# EC.element_to_be_clickable

In [122]:
# page 1

driver.get('https://www.egroup.go.kr/egps/wi/stat/kap/psitnCmpnySumryList.do')
WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div[3]/div[3]/div/form[2]/div/div[1]/div/div/div[2]/a[3]'))
)
bs = BeautifulSoup(driver.page_source, 'lxml')
bs.select('table')

total_elements = []
for i, row in enumerate(bs.select('table')[0].select('tr')):
    if i == 0:
        elements = [tag.text for tag in row.select('th')][:8]
    else:
        elements = [tag.text for tag in row.select('td')][:8]
    
    total_elements.append(elements)

# /html/body/div[2]/div[3]/div[3]/div/form[2]/div/div[1]/div/div/div[2]/a[3] page 2
# /html/body/div[2]/div[3]/div[3]/div/form[2]/div/div[1]/div/div/div[2]/a[11] page 10

driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div[3]/div/form[2]/div/div[1]/div/div/div[2]/a[3]').click()

for page in range(4, 12):
    WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div[3]/div[3]/div/form[2]/div/div[1]/div/div/div[1]/table/tbody/tr[1]/td[3]'))
            )
    bs = BeautifulSoup(driver.page_source, 'lxml')
    bs.select('table')

    for i, row in enumerate(bs.select('table')[0].select('tr')):
        if i == 0:
            elements = [tag.text for tag in row.select('th')][:8]
        else:
            elements = [tag.text for tag in row.select('td')][:8]
        
        total_elements.append(elements)

    if page != 12:
        element = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, f'/html/body/div[2]/div[3]/div[3]/div/form[2]/div/div[1]/div/div/div[2]/a[{page}]'))
            )
        driver.execute_script('arguments[0].click();', element)

In [123]:
len(total_elements)

99

#### ActionChains

연속 동작을 수행하기 위함

ex) control + c
```python
ActionChains(driver).key_down(Keys.CONTROL).send_keys('c').key_up(Keys.CONTROL).perform()
```

##### Drag and Drop

```python
from selenium.webdriver import ActionChains

element = driver.find_element(By.NAME, "source")
target = driver.find_element(By.NAME, "target")

action_chains = ActionChains(driver)
action_chains.drag_and_drop(element, target).perform()
```

#### Others

##### Options
```python
options = webdriver.ChromeOptions()
options.add_argument('window-size=1920,1080')

driver = webdriver.Chrome(executable_path, options=options)
```

##### Alert

경고창 발생 시 이에 수락, 거절 등의 행동을 취할 수 있음

```python
from selenium.webdriver.common.alert import Alert

Alert(driver).accept() # 수락
Alert(driver).dismiss() # 거절
Alert(driver).send_keys(keysToSend=key) # 특정 키를 보낼 수 있음
```

##### Scroll Down
페이지의 최하단으로 이동
```python
driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
```
<br>

특정 태그가 등장할 때까지 이동
```python
from selenium.webdriver import ActionChains

some_tag = driver.find_element_by_id('gorio')
ActionChains(driver).move_to_element(some_tag).perform()
```

##### Minimize/Maximize

```python
driver.minimize_window()
driver.maximize_window()
```

##### Screen Shot
```python
driver.save_screenshot('screenshot.png')
```

In [98]:
driver.save_screenshot('current_page.png')

True

# Practice

## DART

In [124]:
driver.get('https://dart.fss.or.kr/')

# 삼성전자 입력
driver.find_element(
    By.XPATH,
    '/html/body/div[2]/div[1]/div[3]/div/div[2]/form/div[1]/div[2]/div/span[2]/input',
).send_keys('삼성전자')
driver.find_element(
    By.XPATH,
    '/html/body/div[2]/div[1]/div[3]/div/div[2]/form/div[1]/div[2]/div/span[2]/input',
).send_keys(Keys.ENTER)

# 반기보고서 클릭
WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.XPATH, f'/html/body/div[4]/div[2]/div[1]/div[2]/div[2]/form[3]/div/div[1]/table/tbody/tr[6]/td[3]/a'))
).find_element(By.XPATH, '/html/body/div[4]/div[2]/div[1]/div[2]/div[2]/form[3]/div/div[1]/table/tbody/tr[6]/td[3]/a').click()

In [125]:
driver.window_handles

['DBC6D2A5996F1A54D28205A783754A1A', '2CA5A6A0A82424AE34D97E2C30A8845D']

In [106]:
driver.find_element(
    By.XPATH,
    '/html/body/div[3]/div/div[2]/div[1]/div[2]/ul/ul/li[5]/ul/li[2]/ul/li[1]/a'
).click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div/div[2]/div[1]/div[2]/ul/ul/li[5]/ul/li[2]/ul/li[1]/a"}
  (Session info: chrome=127.0.6533.120); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00628923+23283]
	(No symbol) [0x005EE934]
	(No symbol) [0x00520733]
	(No symbol) [0x0056326F]
	(No symbol) [0x005634AB]
	(No symbol) [0x0059EE42]
	(No symbol) [0x00584464]
	(No symbol) [0x0059CB8D]
	(No symbol) [0x005841B6]
	(No symbol) [0x00558017]
	(No symbol) [0x0055890D]
	GetHandleVerifier [0x0071A5F3+1013699]
	GetHandleVerifier [0x00723E4C+1052700]
	GetHandleVerifier [0x0071D4B4+1025668]
	GetHandleVerifier [0x0064EA2B+179195]
	(No symbol) [0x005F6833]
	(No symbol) [0x005F3198]
	(No symbol) [0x005F3337]
	(No symbol) [0x005EB4BE]
	BaseThreadInitThunk [0x75E17BA9+25]
	RtlInitializeExcepti

In [126]:
driver.switch_to.window(driver.window_handles[-1])

In [127]:
driver.find_element(
    By.XPATH,
    '/html/body/div[3]/div/div[2]/div[1]/div[2]/ul/ul/li[5]/ul/li[2]/ul/li[1]/a'
).click()

In [128]:
(
    BeautifulSoup(driver.page_source, 'lxml')
    .select('table')
)

[]

In [129]:
driver.close()

In [130]:
driver.switch_to.window(driver.window_handles[-1])

In [135]:
bs = BeautifulSoup(requests.get('https://dart.fss.or.kr/dsaf001/main.do?rcpNo=20240814003284').text, 'lxml')

In [136]:
# https://dart.fss.or.kr/report/viewer.do?rcpNo=20240814003284&dcmNo=10070946&eleId=20&offset=284259&length=22788&dtd=dart4.xsd
bs

<!DOCTYPE html>
<html lang="ko" xml:lang="ko" xmlns="http://www.w3.org/1999/xhtml">
<head>
<title>삼성전자/반기보고서/2024.08.14</title>
<meta charset="utf-8"/>
<meta content="width=device-width, user-scaleable=no" name="viewport"/>
<meta content="ie=edge" http-equiv="X-UA-Compatible"/>
<script src="/resource/js/jquery-3.3.1.min.js" type="text/javascript"></script>
<link href="/js/jquery-ui/jquery-ui.min.css" rel="stylesheet"/>
<script src="/js/jquery-ui/jquery-ui.min.js" type="text/javascript"></script>
<!-- 2011.11.01 ext 2.3 -->
<!--[if lte IE 8]><link rel="stylesheet" type="text/css" href="/js/ext-main/resources/css/ext-all-ie8.css" /><![endif]-->
<script src="/js/ext-main/adapter/ext/ext-base.js" type="text/javascript"></script>
<script src="/js/ext-main/ext-all.js" type="text/javascript"></script>
<!-- x-xeries js libraries  -->
<script src="/js/xjs.js?ver=1.18" type="text/javascript"></script>
<!-- application js libraries -->
<script src="/js/jsval.js" type="text/javascript"></script>
<